In [6]:
import os
import json

base = "/Users/kevin/Downloads/SecRL/secgym/env/questions/min_overlap/test"

count = 0
for file in os.listdir(base):
    if file.endswith(".json"):
        with open(os.path.join(base, file), "r") as f:
            data = json.load(f)
            
        print(file)
        for i, q in enumerate(data):
            if len(q['context']) < 50:
                print(i)
                print("||", q['context'], "||")
                print("Question", q['question'])
                # print(q['answer'])

                count += 1
        print("-" * 50) 

print(count)

        

incident_38_qa_incident_o1-ga_c42.json
0
|| {'host': 'unspecified'} ||
Question An unexpected behavior was observed by a process that ran with no command line arguments. Can you identify the IP address associated with this activity?
--------------------------------------------------
incident_34_qa_incident_o1-ga_c42.json
10
||  ||
Question Can you identify the command line associated with the suspicious PowerShell activity observed on the machine?
39
|| {'Security Incident': 'Automated investigation started manually involving an uncommon file being created and added to a Run Key.', 'Start Alert': {'Time': '2024-07-04 22:35:00.400749+00:00', 'Name': 'Automated investigation started manually', 'Description': 'User u141 (u141@ash.alpineskihouse.co) initiated an automated investigation on host `vnevado-win10e.vnevado.alpineskihouse.co`.'}, 'Connected Entities': [{'Type': 'host', 'Field': 'HostName', 'Value': 'vnevado-win10e'}, {'Type': 'host', 'Field': 'AadDeviceId', 'Value': '02b8f519-112

In [10]:
import os
import json
import re

# Define base directories
base = "/Users/kevin/Downloads/SecRL/secgym/env/questions/min_overlap/test"
new_base = "/Users/kevin/Downloads/SecRL/secgym/env/questions/min_overlap/test_filtered"

# Ensure new folder exists
os.makedirs(new_base, exist_ok=True)

# Dictionary to store incident IDs and their question keys
incident_questions = {}

# Track removed questions count
removed_count = 0

for file in os.listdir(base):
    if file.endswith(".json"):
        file_path = os.path.join(base, file)

        with open(file_path, "r") as f:
            data = json.load(f)

        # Extract incident ID from filename (e.g., "incident_34_qa_incident_o1-ga_c42.json" → "34")
        match = re.search(r"incident_(\d+)", file)
        if not match:
            continue  # Skip if no match
        incident_id = match.group(1)

        # Initialize storage for retained and removed question keys
        retained_keys = []
        removed_keys = []
        filtered_data = []

        for q in data:
            key = f"{q['start_alert']}-{q['end_alert']}"

            # Remove questions with context < 50
            if len(q['context']) < 50:
                removed_keys.append(key)
                removed_count += 1
                continue
            
            # Remove specific question "48-108" for incident_39
            if incident_id == "39" and key == "48-108":
                removed_keys.append(key)
                removed_count += 1
                continue

            filtered_data.append(q)
            retained_keys.append(key)

        # Store retained and removed question keys
        incident_questions[incident_id] = removed_keys

        # Save the filtered data to the new folder
        new_file_path = os.path.join(new_base, file)
        with open(new_file_path, "w") as f:
            json.dump(filtered_data, f, indent=4)

# Print summary
print(f"Total questions removed: {removed_count}")
print(f"Filtered files saved to: {new_base}")
print("Incident Question Keys Mapping (Retained & Removed):")
print(json.dumps(incident_questions, indent=4))

Total questions removed: 10
Filtered files saved to: /Users/kevin/Downloads/SecRL/secgym/env/questions/min_overlap/test_filtered
Incident Question Keys Mapping (Retained & Removed):
{
    "38": [
        "14-15"
    ],
    "34": [
        "25-10",
        "0-23",
        "3-0"
    ],
    "5": [
        "131-110",
        "100-149"
    ],
    "39": [
        "48-108",
        "138-95"
    ],
    "134": [
        "13-11"
    ],
    "322": [
        "25-23"
    ],
    "166": [],
    "55": []
}


In [9]:
removed_questions = {
    "38": [
        "14-15"
    ],
    "34": [
        "25-10",
        "0-23",
        "3-0"
    ],
    "5": [
        "131-110",
        "100-149"
    ],
    "39": [
        "48-108",
        "138-95"
    ],
    "134": [
        "13-11"
    ],
    "322": [
        "25-23"
    ],
    "166": [],
    "55": []
}

import json

def remove_questions(data_folder):
    incidents = [5, 34, 38, 39, 55, 134, 166, 322]

    for iid in incidents:

        file_name = f"{data_folder}/agent_incident_{iid}.json"
        with open(file_name, "r") as f:
            data = json.load(f)
        
        removed_keys = removed_questions[str(iid)]
        filtered_data = [q for q in data if q['nodes'] not in removed_keys]
        # save to it
        with open(file_name, "w") as f:
            json.dump(filtered_data, f, indent=4)

        env_file = f"{data_folder}/env_incident_{iid}.json"

        with open(env_file, "r") as f:
            data = json.load(f)
        
        filtered_data = [q for q in data if f"{q['question']['start_alert']}-{q['question']['end_alert']}" not in removed_keys]
        # save to it
        with open(env_file, "w") as f:
            json.dump(filtered_data, f, indent=4)





In [10]:
import os

for folder in os.listdir("/Users/kevin/Downloads/SecRL/secgym/final_results"):
    if folder not in ["halfway", "log_level", "PromptSauceAgent_o1-mini_c88_alert_level_t0_s15_trial1", "PromptSauceAgent_o1-mini_c87_alert_level_t0_s15_trial1", "BaselineAgent_o1-mini_c86_alert_level_t0_s15_trial1"]:
        remove_questions(f"/Users/kevin/Downloads/SecRL/secgym/final_results/{folder}")